In [186]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.linear_model as sk
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from random import randint
from IPython.display import display
from sklearn.cluster import KMeans

In [187]:
data = pd.read_csv("/home/shinigami/Downloads/haberman.data",header=None)
data_df = pd.DataFrame(data)
data_df.dropna(inplace=True)
data_df.columns = ['Age','Year','Nodes','Survival_status']
data_df['Survival_status'] = data_df['Survival_status'].astype(int)
data_df['Survival_status'] = data_df['Survival_status'].map({2:0 , 1:1})
data_df.drop_duplicates(inplace=True)

In [188]:
def class_percentage(p,n):
    a,b = list(p['Survival_status']).count(1),list(p['Survival_status']).count(0)
    return a*100/n,b*100/n

In [189]:
class0_df = data_df[data_df['Survival_status']!=1]
class0_df.reset_index(drop=True)
class1_df = data_df[data_df['Survival_status']!=0]
class1_df.reset_index(drop=True)
print()

In [190]:
class0_df['Survival_status'].value_counts()

0    79
Name: Survival_status, dtype: int64

In [191]:
original_data = data_df.copy(deep=True)
original_data_class0 = class0_df.copy(deep=True)
original_data_class1 = class1_df.copy(deep=True)    
class0_df.head(4)

,Age,Year,Nodes,Survival_status
7,34,59,0,0
8,34,66,9,0
24,38,69,21,0
34,39,66,0,0


In [192]:
def flip(a):
    n = len(a)
    p = int(n*0.9)
    for i in range(p):
        rand = randint(0,n-1)
        a['Survival_status'][a.index[rand]] = int(not a['Survival_status'][a.index[rand]])

In [193]:
flip(class0_df)
print(class_percentage(class0_df,len(class0_df)))

/home/shinigami/anaconda3/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(44.30379746835443, 55.69620253164557)


In [194]:
flip(class1_df)
print(class_percentage(class1_df,len(class1_df)))

/home/shinigami/anaconda3/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(59.523809523809526, 40.476190476190474)


# OriginalData Train test split

In [195]:
x_train,x_test,y_train,y_test = train_test_split(data_df.loc[:,data_df.columns!='Survival_status'],data_df['Survival_status'],test_size = 0.38)

# Mislabeled Data 

In [196]:
Mislabeled = class0_df.add(class1_df,fill_value=0)

In [197]:
data_df['Survival_status'].value_counts()

1    210
0     79
Name: Survival_status, dtype: int64

In [198]:
Mislabeled['Survival_status'].value_counts()

1.0    160
0.0    129
Name: Survival_status, dtype: int64

In [199]:
print(177/306 , 193/306 , 48/306)

0.5784313725490197 0.630718954248366 0.1568627450980392


In [200]:
X_train,X_test,Y_train,Y_test = train_test_split(Mislabeled.loc[:,Mislabeled.columns!='Survival_status'],Mislabeled['Survival_status'],test_size = 0.38)

In [201]:
Y_train.unique()

array([0., 1.])

# Model Training

In [202]:
rf = RandomForestClassifier()
rf.fit(X_train,Y_train)
predict = rf.predict(x_test)
accuracy_score(y_test,predict)

/home/shinigami/anaconda3/envs/myenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.45454545454545453

# Original_Model

In [203]:
rf_original = RandomForestClassifier()
rf_original.fit(x_train,y_train)
predict_ori = rf_original.predict(x_test)
accuracy_score(y_test,predict_ori)

/home/shinigami/anaconda3/envs/myenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7181818181818181

In [204]:
data_df.describe()

,Age,Year,Nodes,Survival_status
count,289.000000,289.000000,289.000000,289.000000
mean,52.422145,62.906574,4.221453,0.726644
std,10.876915,3.275794,7.325239,0.446455
min,30.000000,58.000000,0.000000,0.000000
25%,44.000000,60.000000,0.000000,0.000000
50%,52.000000,63.000000,1.000000,1.000000
75%,60.000000,66.000000,5.000000,1.000000
max,83.000000,69.000000,52.000000,1.000000


# Clustering

In [205]:
cl = KMeans(n_clusters=2)
cl.fit(Mislabeled.loc[:,Mislabeled.columns!='Survival_status'])
predict = cl.labels_

In [210]:
print(list(predict).count(0)*100/289,list(predict).count(1)*100/289)
Mislabeled.reset_index(drop=True,inplace=True)

45.32871972318339 54.67128027681661


In [211]:
for i in range(len(Mislabeled)):
    if predict[i]!=Mislabeled['Survival_status'][i]:
        Mislabeled['Survival_status'][i] = np.NaN        

In [213]:
Mislabeled['Survival_status'].value_counts()

1.0    83
0.0    54
Name: Survival_status, dtype: int64

In [214]:
Mislabeled.dropna(inplace=True)

In [215]:
Mislabeled.shape

(137, 4)

In [216]:
x1,x2,y1,y2 = train_test_split(Mislabeled.loc[:,Mislabeled.columns!='Survival_status'],Mislabeled['Survival_status'],test_size = 0.38)

In [217]:
rf_final = RandomForestClassifier()
rf_final.fit(x1,y1)
p1 = rf_final.predict(x_test)
accuracy_score(y_test,p1)

/home/shinigami/anaconda3/envs/myenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.4909090909090909